# Baseline RAG LLM 2

自分なりのRAGを組んで実行する

In [1]:
import os
import pandas as pd
import openai
import datetime
import tiktoken
from sudachipy import tokenizer
from sudachipy import dictionary
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [2]:
# OpenAI APIキーを設定
openai.api_key = input()

In [3]:
novel_file_path = "../data/novels_preprocess/works/"

In [4]:
# .txt ファイルを読み込み、ドキュメントをリスト化
documents = []
for filename in os.listdir(novel_file_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(novel_file_path, filename)
        loader = TextLoader(file_path, encoding="utf-8")
        documents.extend(loader.load())

In [5]:
documents

[Document(metadata={'source': '../data/novels_preprocess/works/カインの末裔.txt'}, page_content='長い影を地にひいて、痩馬の手綱を取りながら、彼れは黙りこくって歩いた。大きな汚い風呂敷包と一緒に、章魚のように頭ばかり大きい赤坊をおぶった彼れの妻は、少し跛脚をひきながら三、四間も離れてその跡からとぼとぼとついて行った。\n北海道の冬は空まで逼っていた。蝦夷富士といわれるマッカリヌプリの麓に続く胆振の大草原を、日本海から内浦湾に吹きぬける西風が、打ち寄せる紆濤のように跡から跡から吹き払っていった。寒い風だ。見上げると八合目まで雪になったマッカリヌプリは少し頭を前にこごめて風に歯向いながら黙ったまま突立っていた。昆布岳の斜面に小さく集った雲の塊を眼がけて日は沈みかかっていた。草原の上には一本の樹木も生えていなかった。心細いほど真直な一筋道を、彼れと彼れの妻だけが、よろよろと歩く二本の立木のように動いて行った。\n二人は言葉を忘れた人のようにいつまでも黙って歩いた。馬が溺りをする時だけ彼れは不性無性に立どまった。妻はその暇にようやく追いついて背の荷をゆすり上げながら溜息をついた。馬が溺りをすますと二人はまた黙って歩き出した。\n「ここらおやじ（熊の事）が出るずら」\n四里にわたるこの草原の上で、たった一度妻はこれだけの事をいった。慣れたものには時刻といい、所柄といい熊の襲来を恐れる理由があった。彼れはいまいましそうに草の中に唾を吐き捨てた。\n草原の中の道がだんだん太くなって国道に続く所まで来た頃には日は暮れてしまっていた。物の輪郭が円味を帯びずに、堅いままで黒ずんで行くこちんとした寒い晩秋の夜が来た。\n着物は薄かった。そして二人は餓え切っていた。妻は気にして時々赤坊を見た。生きているのか死んでいるのか、とにかく赤坊はいびきも立てないで首を右の肩にがくりと垂れたまま黙っていた。\n国道の上にはさすがに人影が一人二人動いていた。大抵は市街地に出て一杯飲んでいたのらしく、行違いにしたたか酒の香を送ってよこすものもあった。彼れは酒の香をかぐと急にえぐられるような渇きと食欲とを覚えて、すれ違った男を見送ったりしたが、いまいましさに吐き捨てようとする唾はもう出て来なかった。糊のように粘ったものが唇の

In [6]:
# テキストを分割するためのテキストスプリッターを定義
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(documents)

Created a chunk of size 571, which is longer than the specified 500
Created a chunk of size 738, which is longer than the specified 500
Created a chunk of size 692, which is longer than the specified 500
Created a chunk of size 1163, which is longer than the specified 500
Created a chunk of size 765, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 697, which is longer than the specified 500
Created a chunk of size 559, which is longer than the specified 500
Created a chunk of size 519, which is longer than the specified 500
Created a chunk of size 1205, which is longer than the specified 500
Created a chunk of size 553, which is longer than the specified 500
Created a chunk of size 510, which is longer than the specified 500
Created a chunk of size 553, which is longer than the specified 500
Created a chunk of size 574, which is longer than the specified 500
Created a chunk of size 543, which is longer t

In [7]:
for i in range(len(split_docs)):
    print(split_docs[i].page_content)

長い影を地にひいて、痩馬の手綱を取りながら、彼れは黙りこくって歩いた。大きな汚い風呂敷包と一緒に、章魚のように頭ばかり大きい赤坊をおぶった彼れの妻は、少し跛脚をひきながら三、四間も離れてその跡からとぼとぼとついて行った。
北海道の冬は空まで逼っていた。蝦夷富士といわれるマッカリヌプリの麓に続く胆振の大草原を、日本海から内浦湾に吹きぬける西風が、打ち寄せる紆濤のように跡から跡から吹き払っていった。寒い風だ。見上げると八合目まで雪になったマッカリヌプリは少し頭を前にこごめて風に歯向いながら黙ったまま突立っていた。昆布岳の斜面に小さく集った雲の塊を眼がけて日は沈みかかっていた。草原の上には一本の樹木も生えていなかった。心細いほど真直な一筋道を、彼れと彼れの妻だけが、よろよろと歩く二本の立木のように動いて行った。
二人は言葉を忘れた人のようにいつまでも黙って歩いた。馬が溺りをする時だけ彼れは不性無性に立どまった。妻はその暇にようやく追いついて背の荷をゆすり上げながら溜息をついた。馬が溺りをすますと二人はまた黙って歩き出した。
「ここらおやじ（熊の事）が出るずら」
「ここらおやじ（熊の事）が出るずら」
四里にわたるこの草原の上で、たった一度妻はこれだけの事をいった。慣れたものには時刻といい、所柄といい熊の襲来を恐れる理由があった。彼れはいまいましそうに草の中に唾を吐き捨てた。
草原の中の道がだんだん太くなって国道に続く所まで来た頃には日は暮れてしまっていた。物の輪郭が円味を帯びずに、堅いままで黒ずんで行くこちんとした寒い晩秋の夜が来た。
着物は薄かった。そして二人は餓え切っていた。妻は気にして時々赤坊を見た。生きているのか死んでいるのか、とにかく赤坊はいびきも立てないで首を右の肩にがくりと垂れたまま黙っていた。
国道の上にはさすがに人影が一人二人動いていた。大抵は市街地に出て一杯飲んでいたのらしく、行違いにしたたか酒の香を送ってよこすものもあった。彼れは酒の香をかぐと急にえぐられるような渇きと食欲とを覚えて、すれ違った男を見送ったりしたが、いまいましさに吐き捨てようとする唾はもう出て来なかった。糊のように粘ったものが唇の合せ目をとじ付けていた。
内地ならば庚申塚か石地蔵でもあるはずの所に、真黒になった一丈もありそうな標示杭が斜めになって立っていた。そこまで来ると干魚をやく香が

In [8]:
# OpenAIの埋め込みモデルを使ってドキュメントをベクトル化
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)

C:\Users\tatsu\AppData\Local\Temp\ipykernel_40652\900527064.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


In [9]:
# FAISSでベクトルストアを作成
vectorstore = FAISS.from_documents(split_docs, embedding)

In [10]:
# SudachiPyの初期設定
tokenizer_obj = dictionary.Dictionary().create()

In [11]:
# SudachiPyを使用して名詞・動詞・形容詞のみを抽出する関数
def extract_relevant_words(query):
    mode = tokenizer.Tokenizer.SplitMode.C  # モードを指定
    tokens = tokenizer_obj.tokenize(query, mode)
    
    # 名詞、動詞、形容詞のみを抽出
    relevant_words = []
    for token in tokens:
        pos = token.part_of_speech()[0]  # 品詞情報を取得
        if pos in ["名詞", "動詞", "形容詞"]:
            relevant_words.append(token.surface())  # 単語の表層形を取得
    
    return " ".join(relevant_words)

In [13]:
# 6. 回答を50トークン以内に制限し、引用を含むプロンプトを作成
prompt_template = """あなたは正確性の高いQAシステムです。
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
以下のルールに従って回答してください。:
1. 回答内で指定されたコンテキストを直接参照しないでください。
2. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。
3. 回答は50トークン以内で簡潔に回答してください。
4. コンテキストから具体的な回答ができない場合は「分かりません」と回答してください。

コンテキスト: {context}
質問: {question}
回答:"""
prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])


In [14]:
# OpenAIの言語モデルを設定（ここではGPT-3を使用）
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=openai.api_key)

C:\Users\tatsu\AppData\Local\Temp\ipykernel_40652\1591884372.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-4o", openai_api_key=openai.api_key)


In [15]:
# Retrieverとしてベクトルストアを設定し、top_kを設定
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [16]:
# 検索用のQAチェーンを構築
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # "stuff" モードはシンプルに関連ドキュメントをまとめて渡すモード
    retriever=retriever,
    return_source_documents=True,  # 検索結果としてソースドキュメントを返す
    chain_type_kwargs={"prompt": prompt}
)

## 質問ファイルを読み込んでQ&Aを作成

In [17]:
# 提供されたCSVファイルを読み込み
query_df = pd.read_csv("../data/query.csv", encoding="utf-8")

In [18]:
query_df.head()

,index,problem
0,1,競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？
1,2,骸骨男の正体は誰ですか？作中で言及されている氏名で答えること。
2,3,骸骨男はバスの中に足跡を一切残さずにどうやって抜け出しましたか？
3,4,殺人罪で裁判にかけられた兄が登場しますが、その理由は何ですか？
4,5,小説「芽生」で出てくる国名は何種類ですか？


In [19]:
answers = []
evidences = []

for _, row in query_df.iterrows():
    print(_)
    query = row["problem"]
    processed_query = extract_relevant_words(query)  # 質問を前処理して名詞・動詞・形容詞だけを抽出
    print(query)
    print(processed_query)
    result = qa_chain({"query": processed_query})
    answer = result["result"]
    print(answer)
    evidence = result["source_documents"][0].page_content # 証拠部分を抽出
    answers.append(answer)
    evidences.append(evidence)

0
競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？
競漕 会 三 前 レースコース 結果 農科 文科 秒数 差 あっ


C:\Users\tatsu\AppData\Local\Temp\ipykernel_40652\3660868621.py:10: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa_chain({"query": processed_query})


分かりません。
1
骸骨男の正体は誰ですか？作中で言及されている氏名で答えること。
骸骨 男 正体 作中 言及 さ いる 氏名 答える こと
分かりません。
2
骸骨男はバスの中に足跡を一切残さずにどうやって抜け出しましたか？
骸骨 男 バス 中 足跡 一切 残さ やっ 抜け出し
分かりません。
3
殺人罪で裁判にかけられた兄が登場しますが、その理由は何ですか？
殺人罪 裁判 かけ 兄 登場 し 理由
兄が殺人罪で裁判にかけられたのは、恋人と共に情死を図り、恋人を短刀で殺したためです。
4
小説「芽生」で出てくる国名は何種類ですか？
小説 芽生 出 くる 国名 種類
分かりません。
5
武男と浪子のうち、どちらが野外での楽しみをより積極的に表現していますか？
武男 浪子 うち 野外 楽しみ 積極的 表現 し い
はい、武男と浪子は野外での景色や時間を楽しんでいる様子が積極的に表現されています。
6
小説「芽生」で、主人公が東京から小諸に帰る時期から、実際に引越しが完了するまでの間はだいたいどのぐらいの日数だと考えられますか？
小説 芽生 主人公 東京 小諸 帰る 時期 実際 引越し 完了 する 間 だいたい 日数 考え
分かりません。
7
「競漕」の中で、文科のボート部メンバーが実際に酒を飲むシーンは何回か？
競漕 中 文科 ボート 部 メンバー 実際 酒 飲む シーン 何回
1回です。
8
文中に登場するカタカナ表記された北海道の地名を全て挙げてください。
文中 登場 する カタカナ 表記 さ 北海道 地名 全て 挙げ ください
分かりません。
9
兄が情死を試みた時点から、実際に死亡するまでおよそ何年経過したと考えられますか？
兄 情死 試み 時点 実際 死亡 する 何 年 経過 し 考え
分かりません。
10
お房が小諸を出発した日に一緒に出かけた友達は誰でしたか？
房 小諸 出発 し 日 一緒 出かけ 友達
分かりません。
11
サーカス団の楽屋にいるライオンの数は合計何匹ですか？
サーカス 楽屋 いる ライオン 数 合計 何
分かりません。
12
死生に関するいくつかの断想に登場する二人の兄弟のうち、より健康だったのはどちらですか？
死生 関する いくつ 断想 登場 する 二人 兄弟 うち 健康
分かりません。
13
笠原正一君が見た恐ろしい夢の内容は何で

In [20]:
# DataFrameに回答と証拠を追加
query_df['full_answer'] = answers
query_df['evidence'] = evidences

In [21]:
replace_dict = {
        "\n": "",
        "\r": "",
    }

query_df = query_df.replace(
        {"evidence": replace_dict},
        regex=True
    )

In [22]:
query_df.head(20)

,index,problem,full_answer,evidence
0,1,競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？,分かりません。,いよいよ季節に入ったので高商、明治という工合に次ぎ次ぎ競漕会が行われた。そうなって来ると勝敗...
1,2,骸骨男の正体は誰ですか？作中で言及されている氏名で答えること。,分かりません。,そいつが、立ちあがって、こちらをむきました。その顔！ やっぱりそうでした。骸骨です。骸骨の顔...
2,3,骸骨男はバスの中に足跡を一切残さずにどうやって抜け出しましたか？,分かりません。,骸骨男の左手が、首のほそびきにかかっていました。そうしなければ、首がしまって死んでしまうから...
3,4,殺人罪で裁判にかけられた兄が登場しますが、その理由は何ですか？,兄が殺人罪で裁判にかけられたのは、恋人と共に情死を図り、恋人を短刀で殺したためです。,どんな刑が言い渡されたのか、詳細は知らない。当時日本の裁判官は、人情がらみの犯罪を裁く場合、...
4,5,小説「芽生」で出てくる国名は何種類ですか？,分かりません。,ラッパを吹き鼓を鳴らして名を売ることをせざれば、知らざる者は名をだに聞かざれど、知れる者はそ...
5,6,武男と浪子のうち、どちらが野外での楽しみをより積極的に表現していますか？,はい、武男と浪子は野外での景色や時間を楽しんでいる様子が積極的に表現されています。,と武男はしいて浪子を押しとめ、ハンケチ包みの蕨を草の上にさし置き、急ぎ足に坂を下りて見えずな...
6,7,小説「芽生」で、主人公が東京から小諸に帰る時期から、実際に引越しが完了するまでの間はだいたい...,分かりません。,二週間ばかり経ったところで、大久保の植木屋から手紙を受取った。見ると、月の末まで待たなければ...
7,8,「競漕」の中で、文科のボート部メンバーが実際に酒を飲むシーンは何回か？,1回です。,皆はかなり好い気持であった。そしていつもよりは活気づいて艇庫に船を蔵めた。夕飯には褒賞の意味...
8,9,文中に登場するカタカナ表記された北海道の地名を全て挙げてください。,分かりません。,誠におはずかしき事に候えどもどうやらいたし候節はさびしさ悲しさのやる瀬なく早く早く早く御目に...
9,10,兄が情死を試みた時点から、実際に死亡するまでおよそ何年経過したと考えられますか？,分かりません。,一二月二九日。 予想したように、死んだ男の話はなかなか聴きでがあった。この家族は四人である―...


In [25]:
# LLMを使って要約を行う関数
def summarize_answer(problem: str, full_answer: str, evidence: str) -> str:

    summarize_prompt = PromptTemplate(
        input_variables=["problem", "full_answer", "evidence"],
        template=
            """以下の質問に対する回答の文章を証拠を元に50文字以内に収まるように簡潔に答え直してください。
                f"質問: {problem}\n\n"
                f"回答: {full_answer}\n"
                f"証拠: {evidence}\n"
            """
    )
    chain = summarize_prompt | llm

    response = chain.invoke(
        {"problem": problem, "full_answer": full_answer, "evidence": evidence}
    )
    return response.content

In [23]:
# tiktokenとgpt-4のトークナイザーを取得
enc = tiktoken.encoding_for_model("gpt-4-2024-08-06")

# query_df の "answer" 列のトークン数を計算し、50トークンを超える場合は要約を行う関数
def check_and_summarize_answers(query_df: pd.DataFrame) -> pd.DataFrame:
    def summarize_if_needed(problem: str, full_answer: str, evidence: str) -> str:
        # トークン数を計算
        token_count = len(enc.encode(full_answer))
        print(token_count)
        
        # トークン数が50を超えた場合は要約する
        if token_count > 50:
            # LLMを使って要約
            summarized_answer = summarize_answer(problem, full_answer, evidence)
            return summarized_answer
        return full_answer

    # "answer" 列に対して処理を適用
    query_df["answer"] = query_df["full_answer"]
    for i in range(len(query_df.index)):
        query_df["answer"][i] = summarize_if_needed(query_df["problem"][i], query_df["full_answer"][i], query_df["evidence"][i])
    return query_df

In [ ]:
query_df = check_and_summarize_answers(query_df)

In [ ]:
query_df.head(20)

In [28]:
# 必要な列（id, answer, evidence）をヘッダなしでCSVに書き出し
query_df[['index', 'answer', 'evidence']].to_csv(
    "../submit/predictions.csv",
    index=False,
    header=False,
    encoding="utf-8-sig"
)

In [29]:
# backup
dt_now = datetime.datetime.now()
ymdm = dt_now.strftime("%Y%m%d-%H%M")

query_df[['index', 'problem', 'full_answer', 'answer', 'evidence']].to_csv(
    f"../submit/{ymdm}_predictions.csv",
    index=False,
    header=True,
    encoding="utf-8-sig"
)